Please first do the first tutorial to get familia with the definition of robots and matrics.

# Tutorial 2: Robot Kinematics

In [2]:
from sympy import symbols, pi

from robot import Robot, JointType

from robot import Vector, Matrix

In [9]:
t1, t2, t3 = symbols('t1 t2 t3')
l1, l2, l3 = symbols('l1 l2 l3')
g = symbols('g')

In [5]:
dh = [
    [0, 0, 0, t1],
    [l1, pi/2, 0, t2],
    [l2,0,0,t3],
    [l3, 0, 0, 0]
]

robot = Robot.from_dh_parameters(dh, [JointType.ROTATIONAL, JointType.ROTATIONAL, JointType.ROTATIONAL])

robot

Robot[
 - BASE joint	 [a: 0.0, alpha: 0.0, d: 0, theta: 0]
 - ROTATIONAL joint	 [a: 0, alpha: 0, d: 0, theta: t1]
 - ROTATIONAL joint	 [a: l1, alpha: pi/2, d: 0, theta: t2]
 - ROTATIONAL joint	 [a: l2, alpha: 0, d: 0, theta: t3]
 - END_EFFECTOR	 [a: l3, alpha: 0, d: 0, theta: 0]
]

Given a robot we can easily calculate all transformation matrices:

In [6]:
robot.get_transformations()

Matrix[
	cos(t1),    -sin(t1),    0,    0,    
	sin(t1),    cos(t1),    0,    0,    
	0,    0,    1,    0,    
	0,    0,    0,    1,    
]
Matrix[
	cos(t2),    -sin(t2),    0,    l1,    
	0,    0,    -1,    0,    
	sin(t2),    cos(t2),    0,    0,    
	0,    0,    0,    1,    
]
Matrix[
	cos(t3),    -sin(t3),    0,    l2,    
	sin(t3),    cos(t3),    0,    0,    
	0,    0,    1,    0,    
	0,    0,    0,    1,    
]
Matrix[
	1,    0,    0,    l3,    
	0,    1,    0,    0,    
	0,    0,    1,    0,    
	0,    0,    0,    1,    
]


We can also get the position of the endeffector:

In [7]:
p_e = (robot.get_end_transformation() @ Vector([0, 0, 0, 1])).simplify()

p_e

Vector[	(l1 + l2*cos(t2) + l3*cos(t2 + t3))*cos(t1),  (l1 + l2*cos(t2) + l3*cos(t2 + t3))*sin(t1),  l2*sin(t2) + l3*sin(t2 + t3),  1,  ]

We can calculate both the static and the dynamic kinematics of the robot. To calculate the static dynamics simply call:

In [8]:
ws, vs, fs, ns, ts = robot.cal_static_kinematics(True, True)

joint 1:
 - w: Vector[	0,  0,  dot_t1,  ]
 - v: Vector[	0,  0,  0,  ]
joint 2:
 - w: Vector[	dot_t1*sin(t2),  dot_t1*cos(t2),  dot_t2,  ]
 - v: Vector[	0,  0,  -dot_t1*l1,  ]
joint 3:
 - w: Vector[	dot_t1*sin(t2 + t3),  dot_t1*cos(t2 + t3),  dot_t2 + dot_t3,  ]
 - v: Vector[	dot_t2*l2*sin(t3),  dot_t2*l2*cos(t3),  -dot_t1*(l1 + l2*cos(t2)),  ]
joint 4:
 - w: Vector[	dot_t1*sin(t2 + t3),  dot_t1*cos(t2 + t3),  dot_t2 + dot_t3,  ]
 - v: Vector[	dot_t2*l2*sin(t3),  dot_t2*l2*cos(t3) + l3*(dot_t2 + dot_t3),  -dot_t1*(l1 + l2*cos(t2) + l3*cos(t2 + t3)),  ]

backwards:

joint 3:
 - f: Vector[	F1,  F2,  F3,  ]
 - n: Vector[	N1,  -F3*l3 + N2,  F2*l3 + N3,  ]
simplified 3:
 - f: Vector[	F1,  F2,  F3,  ]
 - n: Vector[	N1,  -F3*l3 + N2,  F2*l3 + N3,  ]
joint 2:
 - f: Vector[	F1*cos(t3) - F2*sin(t3),  F1*sin(t3) + F2*cos(t3),  F3,  ]
 - n: Vector[	N1*cos(t3) - (-F3*l3 + N2)*sin(t3),  -F3*l2 + N1*sin(t3) + (-F3*l3 + N2)*cos(t3),  F2*l3 + N3 + l2*(F1*sin(t3) + F2*cos(t3)),  ]
 - tau: F2*l3 + N3 + l2

Before we can calculate the dynamic kinematics, we have to define the centers of mass as well as the gravity vector:

In [11]:
cof_points = [Vector([0, 0, 0]), Vector([l1 , 0, 0]), Vector([0, 0, 0])]
robot.add_cof(cof_points)
# gravity
g_vec = Vector([0, g, 0])

Now we can calculate the dynamic kinematics:

In [12]:
_ = robot.cal_newton_euler(True, True, g_vec)

----------------Forwards Newton Euler----------------



joint 1:
 - w: Vector[	0,  0,  dot_t1,  ]
 - dw: Vector[	0,  0,  ddot_t1,  ]
 - dv: Vector[	g*sin(t1),  g*cos(t1),  0,  ]
simplified 1:
 - w: Vector[	0,  0,  dot_t1,  ]
 - dw: Vector[	0,  0,  ddot_t1,  ]
 - dv: Vector[	g*sin(t1),  g*cos(t1),  0,  ]
joint 1:
 - dvc: Vector[	g*sin(t1),  g*cos(t1),  0,  ]
 - F: Vector[	g*m1*sin(t1),  g*m1*cos(t1),  0,  ]
 - N: Vector[	0,  0,  I_zz1*ddot_t1,  ]
simplified 1:
 - dvc: Vector[	g*sin(t1),  g*cos(t1),  0,  ]
 - F: Vector[	g*m1*sin(t1),  g*m1*cos(t1),  0,  ]
 - N: Vector[	0,  0,  I_zz1*ddot_t1,  ]

joint 2:
 - w: Vector[	dot_t1*sin(t2),  dot_t1*cos(t2),  dot_t2,  ]
 - dw: Vector[	ddot_t1*sin(t2) + dot_t1*dot_t2*cos(t2),  ddot_t1*cos(t2) - dot_t1*dot_t2*sin(t2),  ddot_t2,  ]
 - dv: Vector[	(-dot_t1**2*l1 + g*sin(t1))*cos(t2),  -(-dot_t1**2*l1 + g*sin(t1))*sin(t2),  -ddot_t1*l1 - g*cos(t1),  ]
simplified 2:
 - w: Vector[	dot_t1*sin(t2),  dot_t1*cos(t2),  dot_t2,  ]
 - dw: Vector[	ddot_t1*sin

joint 3:
 - f: Vector[	m3*(ddot_t2*l2*sin(t3) - dot_t1**2*l1*cos(t2 + t3) + dot_t1**2*l2*sin(t2)*sin(t2 + t3) - dot_t1**2*l2*cos(t3) - dot_t2**2*l2*cos(t3) + g*sin(t1)*cos(t2 + t3)),  m3*(ddot_t2*l2*cos(t3) + dot_t1**2*l1*sin(t2 + t3) + dot_t1**2*l2*sin(t2)*cos(t2 + t3) + dot_t1**2*l2*sin(t3) + dot_t2**2*l2*sin(t3) - g*sin(t1)*sin(t2 + t3)),  m3*(-ddot_t1*l1 - ddot_t1*l2*cos(t2) + 2*dot_t1*dot_t2*l2*sin(t2) - g*cos(t1)),  ]
 - n: Vector[	I_xx3*(ddot_t1*sin(t2 + t3) + dot_t1*dot_t2*cos(t2 + t3) + dot_t1*dot_t3*cos(t2 + t3)) - I_yy3*dot_t1*(dot_t2 + dot_t3)*cos(t2 + t3) + I_zz3*dot_t1*(dot_t2 + dot_t3)*cos(t2 + t3),  I_xx3*dot_t1*(dot_t2 + dot_t3)*sin(t2 + t3) - I_yy3*(-ddot_t1*cos(t2 + t3) + dot_t1*dot_t2*sin(t2 + t3) + dot_t1*dot_t3*sin(t2 + t3)) - I_zz3*dot_t1*(dot_t2 + dot_t3)*sin(t2 + t3),  -I_xx3*dot_t1**2*sin(2*t2 + 2*t3)/2 + I_yy3*dot_t1**2*sin(2*t2 + 2*t3)/2 + I_zz3*(ddot_t2 + ddot_t3),  ]
 - tau: 1
simplified 3:
 - f: Vector[	m3*(ddot_t2*l2*sin(t3) - dot_t1**2*l1*cos(t2 + t3) +

KeyboardInterrupt: 

If we do not want the results to be printed to the console we can set the first bool to `False`:

In [ ]:
result = robot.cal_newton_euler(False, True, g_vec)